In [1]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [2]:
train_df = pd.read_csv('train_df.csv', index_col=0)
test_df = pd.read_csv('test_df.csv', index_col=0)

In [11]:
product_df = pd.read_csv('product_df.csv', index_col=0)
product_df = product_df[2:]
product_df.head()

E:\ProgramData\Anaconda3\envs\PythonGPU\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,1,2,3,4,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,review_id,star_rating,star_rating.1,total_votes,total_votes.1,helpful_votes,helpful_votes.1,corpus
1602,3,3.6666666666666665,1.5275252316519468,0.3333333333333333,0.5773502691896258,0.0,0.0,keyboard month featur problems. < br / > < br ...
2696,3,3.6666666666666665,1.5275252316519468,1.3333333333333333,0.5773502691896257,0.6666666666666666,1.1547005383792517,one pad roccat cu surfac metal foil. < br / > ...
3140,1,1.0,0.0,0.0,0.0,0.0,0.0,peel
4318,23,4.086956521739131,1.202763616096502,18.652173913043477,28.128938020305664,14.217391304347826,22.633355356719807,dimens purs i ipad mini im sick god tablet i t...
10581,7,5.0,0.0,2.7142857142857144,3.302235894778249,2.5714285714285716,3.408672412985387,did extens cabl bulk cabl length odd patch cab...


In [3]:
import heapq
def get_top_3(corpus):
    res = ['', '', '']
    words = str(corpus).split()
    words = {w : words.count(w) for w in words}
    
    heap_array = heapq.nlargest(3, words, key=words.get)
    i = 0
    for w in heap_array:
        res[i] = w
        i += 1
        
    return res

In [13]:
product_top_3 = product_df.corpus.apply(get_top_3)

In [14]:
product_top_3

1602               [keyboard, br, /]
2696                      [<, br, /]
3140                      [peel, , ]
4318                 [/, tablet, br]
10581                   [cabl, /, <]
                      ...           
999976732    [stock, batteri, short]
999978079                 [/, >, br]
999992470         [tab4, inch, snug]
999992934      [case, cutout, kindl]
999996025      [item, check, condit]
Name: corpus, Length: 214749, dtype: object

In [15]:
product_df['word_1'] = [item[0] for item in product_top_3.to_list()]
product_df['word_2'] = [item[1] for item in product_top_3.to_list()]
product_df['word_3'] = [item[2] for item in product_top_3.to_list()]

In [21]:
product_df.loc[999992470]

review_id                                                          1
star_rating                                                        5
star_rating.1                                                      0
total_votes                                                        0
total_votes.1                                                      0
helpful_votes                                                      0
helpful_votes.1                                                    0
corpus             tab4 inch snug price hole place stitch case li...
word_1                                                          tab4
word_2                                                          inch
word_3                                                          snug
Name: 999992470, dtype: object

In [17]:
product_df.to_csv('product_df2.csv')

In [26]:
word_df = product_df[['word_1', 'word_2', 'word_3']]
word_df['product_parent'] = word_df.index
word_df.head()

E:\ProgramData\Anaconda3\envs\PythonGPU\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,word_1,word_2,word_3,product_parent
1602,keyboard,br,/,1602
2696,<,br,/,2696
3140,peel,,,3140
4318,/,tablet,br,4318
10581,cabl,/,<,10581


In [29]:
test_df = pd.merge(test_df, word_df, how='inner', on='product_parent')

In [79]:
train_df = pd.merge(train_df, word_df, how='inner', on='product_parent')

In [43]:
import nltk
from nltk.corpus import wordnet as wn
def word_sim(w1, w2):
    if (isinstance(w1, float) or isinstance(w2, float)):
        return 0
    w1_synset = wn.synsets(w1)
    w2_synset = wn.synsets(w2)
    if (len(w1_synset) == 0 or len(w2_synset) == 0):
        return 0
    w1_syn = w1_synset[0]
    w2_syn = w2_synset[0]
    sim = w1_syn.wup_similarity(w2_syn)
    if (sim == None):
        return 0
    return sim

In [47]:
len([x for x in test_df.apply((lambda x: word_sim(x.pos_1, x.word_1)), axis=1) if x == 0]) / len(test_df)

0.5417113988001263

In [36]:
word_sim('kindl', 'book')

0

In [48]:
train_cust_df = pd.read_csv('train_cust_df.csv', index_col=0)
test_cust_df = pd.read_csv('test_cust_df.csv', index_col=0)

E:\ProgramData\Anaconda3\envs\PythonGPU\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,1,2,3,4,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [53]:
corpus = [w.split() for w in train_cust_df.pos_corpus if not isinstance(w, float)]
corpus = corpus + [w.split() for w in train_cust_df.neg_corpus if not isinstance(w, float)]
corpus = corpus + [w.split() for w in product_df.corpus if not isinstance(w, float)]

In [57]:
import gensim 

# build vocabulary and train model
model = gensim.models.Word2Vec(
    corpus,
    size=150,
    window=10,
    min_count=2,
    workers=10,
    iter=10)

In [63]:
model.wv.most_similar(positive='terribl', topn=6)

[('horribl', 0.7745047211647034),
 ('aw', 0.658147394657135),
 ('bad', 0.6169096231460571),
 ('crappi', 0.6089656352996826),
 ('worst', 0.5955891013145447),
 ('suck', 0.5561203956604004)]

In [67]:
def word_vec_sim(w1, w2):
    if (isinstance(w1, float) or isinstance(w2, float)):
        return 0
    if (w1 not in model.wv.vocab or w2 not in model.wv.vocab):
        return 0
    sim = model.wv.similarity(w1=w1, w2=w2)
    if (sim == None):
        return 0
    return sim

In [78]:
len([x for x in test_df.apply((lambda x: word_vec_sim(x.pos_1, x.word_1)), axis=1) if x == 0]) / len(test_df)

0.009156930849384275

In [85]:
len([x for x in train_df.apply((lambda x: word_vec_sim(x.pos_1, x.word_1)), axis=1) if x == 0]) / len(train_df)

0.015045471690205776

In [80]:
def add_sim_cols(df):
    df['pos_1_word_1'] = df.apply((lambda x: word_vec_sim(x.pos_1, x.word_1)), axis=1)
    df['pos_1_word_2'] = df.apply((lambda x: word_vec_sim(x.pos_1, x.word_2)), axis=1)
    df['pos_1_word_3'] = df.apply((lambda x: word_vec_sim(x.pos_1, x.word_3)), axis=1)
    df['pos_2_word_1'] = df.apply((lambda x: word_vec_sim(x.pos_2, x.word_1)), axis=1)
    df['pos_2_word_2'] = df.apply((lambda x: word_vec_sim(x.pos_2, x.word_2)), axis=1)
    df['pos_2_word_3'] = df.apply((lambda x: word_vec_sim(x.pos_2, x.word_3)), axis=1)
    df['pos_3_word_1'] = df.apply((lambda x: word_vec_sim(x.pos_3, x.word_1)), axis=1)
    df['pos_3_word_2'] = df.apply((lambda x: word_vec_sim(x.pos_3, x.word_2)), axis=1)
    df['pos_3_word_3'] = df.apply((lambda x: word_vec_sim(x.pos_3, x.word_3)), axis=1)
    df['neg_1_word_1'] = df.apply((lambda x: word_vec_sim(x.neg_1, x.word_1)), axis=1)
    df['neg_1_word_2'] = df.apply((lambda x: word_vec_sim(x.neg_1, x.word_2)), axis=1)
    df['neg_1_word_3'] = df.apply((lambda x: word_vec_sim(x.neg_1, x.word_3)), axis=1)
    df['neg_2_word_1'] = df.apply((lambda x: word_vec_sim(x.neg_2, x.word_1)), axis=1)
    df['neg_2_word_2'] = df.apply((lambda x: word_vec_sim(x.neg_2, x.word_2)), axis=1)
    df['neg_2_word_3'] = df.apply((lambda x: word_vec_sim(x.neg_2, x.word_3)), axis=1)
    df['neg_3_word_1'] = df.apply((lambda x: word_vec_sim(x.neg_3, x.word_1)), axis=1)
    df['neg_3_word_2'] = df.apply((lambda x: word_vec_sim(x.neg_3, x.word_2)), axis=1)
    df['neg_3_word_3'] = df.apply((lambda x: word_vec_sim(x.neg_3, x.word_3)), axis=1)
    
    return df

In [81]:
test_df = add_sim_cols(test_df)
test_df.head()

,customer_id,product_parent,star_rating,positive,cust_review_count,cust_star_rating_mean,cust_star_rating_std,cust_total_votes_mean,cust_total_votes_std,cust_helpful_votes_mean,...,pos_3_word_3,neg_1_word_1,neg_1_word_2,neg_1_word_3,neg_2_word_1,neg_2_word_2,neg_2_word_3,neg_3_word_1,neg_3_word_2,neg_3_word_3
0,36330222,986428010,5,1,24,4.291667,1.267629,0.541667,1.250362,0.333333,...,0.001941,0.000987,-0.015594,-0.013630,0.006534,0.007653,0.023873,-0.029103,-0.046843,-0.041810
1,24360083,986428010,4,1,23,4.347826,1.191206,1.434783,2.191431,1.000000,...,0.901494,-0.222196,-0.196039,-0.217517,0.918800,0.925143,0.901494,0.919331,0.902017,1.000000
2,28891040,437083384,4,1,10,4.500000,0.707107,0.000000,0.000000,0.000000,...,0.092532,0.313854,0.086877,0.315884,0.196752,0.052950,0.194290,0.054040,0.014332,-0.002904
3,52449052,437083384,3,0,5,3.400000,1.516575,11.800000,20.789420,10.400000,...,0.187860,0.083284,-0.115922,0.023810,0.090283,-0.083403,0.025464,0.084633,-0.092710,0.019767
4,27192976,437083384,4,1,6,4.666667,0.816497,0.500000,1.224745,0.000000,...,0.126235,-0.083963,0.036128,-0.155615,-0.020984,0.021015,-0.042505,0.059809,-0.017769,-0.064662


In [82]:
test_df.to_csv('test_df2.csv')

In [84]:
train_df = add_sim_cols(train_df)
train_df.head()

,customer_id,product_parent,star_rating,positive,cust_review_count,cust_star_rating_mean,cust_star_rating_std,cust_total_votes_mean,cust_total_votes_std,cust_helpful_votes_mean,...,pos_3_word_3,neg_1_word_1,neg_1_word_2,neg_1_word_3,neg_2_word_1,neg_2_word_2,neg_2_word_3,neg_3_word_1,neg_3_word_2,neg_3_word_3
0,1617361,849246716,5,1,5,5.000000,0.000000,0.200000,0.447214,0.000000,...,0.186957,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,53052607,849246716,3,0,5,3.600000,0.547723,0.400000,0.894427,0.400000,...,-0.028547,0.172017,0.084518,0.194290,0.002472,0.006273,0.052950,0.344210,0.307019,0.363920
2,15679577,849246716,5,1,8,3.375000,1.505941,7.625000,19.175412,6.500000,...,0.044000,0.371702,0.000015,0.316641,0.156807,0.052990,0.165214,0.011699,-0.066285,-0.171195
3,16367779,849246716,5,1,9,4.444444,0.527046,0.888889,1.364225,0.777778,...,-0.171195,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,25485198,849246716,5,1,17,5.000000,0.000000,0.235294,0.437237,0.176471,...,0.249734,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [86]:
train_df.to_csv('train_df2.csv')

In [88]:
product_df.loc[861711947]

review_id                                                        319
star_rating                                                  4.78997
star_rating.1                                               0.579872
total_votes                                                  2.40752
total_votes.1                                                13.8315
helpful_votes                                                1.57367
helpful_votes.1                                              12.3927
corpus             work default vcore.. price point reason intel ...
word_1                                                             /
word_2                                                            br
word_3                                                             >
Name: 861711947, dtype: object

In [89]:
train_df[train_df.product_parent == 861711947]

,customer_id,product_parent,star_rating,positive,cust_review_count,cust_star_rating_mean,cust_star_rating_std,cust_total_votes_mean,cust_total_votes_std,cust_helpful_votes_mean,...,pos_3_word_3,neg_1_word_1,neg_1_word_2,neg_1_word_3,neg_2_word_1,neg_2_word_2,neg_2_word_3,neg_3_word_1,neg_3_word_2,neg_3_word_3
25065,1800042,861711947,5,1,5,5.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25066,18744479,861711947,5,1,7,4.428571,1.511858,0.571429,1.133893,0.428571,...,-0.051168,-0.094633,-0.095282,-0.048976,-0.213596,-0.163611,-0.168251,0.134768,0.113645,0.136326
25067,1829828,861711947,5,1,10,5.000000,0.000000,0.800000,0.918937,0.500000,...,0.023595,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25068,45929928,861711947,5,1,7,5.000000,0.000000,0.142857,0.377964,0.142857,...,-0.264425,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25069,9541999,861711947,5,1,5,5.000000,0.000000,0.000000,0.000000,0.000000,...,-0.041398,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25070,13879794,861711947,5,1,13,4.923077,0.277350,1.000000,1.779513,0.692308,...,-0.142240,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25071,47416009,861711947,5,1,7,4.285714,1.496026,1.285714,2.563480,1.000000,...,-0.055888,0.098734,0.065472,0.069314,-0.024104,-0.005220,-0.002534,-0.041952,-0.050361,-0.055888
25072,50599855,861711947,5,1,6,4.666667,0.816497,0.166667,0.408248,0.000000,...,0.010037,0.022546,0.001124,0.010037,-0.086510,-0.063593,-0.064885,-0.211128,-0.205608,-0.226793
25073,11927407,861711947,5,1,11,4.090909,1.221028,0.181818,0.404520,0.090909,...,0.916021,0.143492,0.143456,0.124290,-0.038071,-0.074058,-0.106970,-0.021857,-0.021420,-0.030142
25074,38516601,861711947,5,1,7,5.000000,0.000000,1.285714,1.799471,1.000000,...,0.122874,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
